In [1]:
import cx_Oracle
import pandas as pd
from neuralprophet import NeuralProphet


c:\anaconda3\envs\NeuralProphet\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:

conn =  cx_Oracle.connect(user="pc",password="p201404",dsn="siatchdesa")
query = "select periodo,customer_id,product_id,clase from L_DATOS_ENTRENAMIENTO where rownum <= 100000"
df = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM

# Para concatenar todos los chunks en un solo DataFrame (si tienes suficiente RAM)
df_full = pd.concat(df, ignore_index=True)
conn.close()
print(df_full.shape)
# Imprimir las primeras filas del DataFrame completo
print(df_full.head())
# Imprimir los tipos de datos de las columnas del DataFrame completo
print(df_full.dtypes)
# Imprimir el número de filas y columnas del DataFrame completo
print(f"Número de filas: {df_full.shape[0]}, Número de columnas: {df_full.shape[1]}")

WARNING - (py.warnings._showwarnmsg) - C:\Users\pablo\AppData\Local\Temp\ipykernel_28572\2628659266.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM



(100000, 4)
   PERIODO  CUSTOMER_ID  PRODUCT_ID    CLASE
0   201701        10348       20654  0.00253
1   201701        10223       20654  0.00760
2   201701        10392       20654  0.00000
3   201701        10082       20654  0.00000
4   201701        10416       20654  0.00317
PERIODO          int64
CUSTOMER_ID      int64
PRODUCT_ID       int64
CLASE          float64
dtype: object
Número de filas: 100000, Número de columnas: 4


In [ ]:
# Preprocesar para NeuralProphet: seleccionar una serie temporal (por ejemplo, por customer_id y product_id)
# Seleccionamos una combinación de CUSTOMER_ID y PRODUCT_ID que tenga varias filas
example = df_full.groupby(['CUSTOMER_ID', 'PRODUCT_ID']).size().reset_index(name='count')
example = example[example['count'] > 1].iloc[0]
customer_id = example['CUSTOMER_ID']
product_id = example['PRODUCT_ID']

# Filtramos la serie temporal para esa combinación
df_prophet = df_full[(df_full['CUSTOMER_ID'] == customer_id) & (df_full['PRODUCT_ID'] == product_id)][['PERIODO', 'CLASE']]
df_prophet = df_prophet.rename(columns={'PERIODO': 'ds', 'CLASE': 'y'})

# Si 'ds' es numérico, conviértelo a string o fecha si corresponde
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'].astype(str), format='%Y%m')

# Entrenar el modelo NeuralProphet
model = NeuralProphet()
metrics = model.fit(df_prophet, freq='MS')  # 'MS' para inicio de mes


WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [50.]% of the data.
WARNING - (NP.df_utils._infer_frequency) - Dataframe has multiple frequencies. It will be resampled according to given freq MS. Ignore message if actual frequency is any of the following:  SM, BM, CBM, SMS, BMS, CBMS, BQ, BQS, BA, or, BAS.
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config

Epoch 1000: 100%|██████████| 1000/1000 [00:00<00:00, 66526.62it/s, loss=1.2e-07, v_num=2, MAE=0.0438, RMSE=0.0541, Loss=1.2e-7, RegLoss=0.000]  


AttributeError: 'NeuralProphet' object has no attribute 'save'

In [16]:
import torch

# Guardar solo los pesos y parámetros del modelo
torch.save(model.state_dict(), "modelo_neuralprophet_state.pth")

In [17]:
# Cargo los datos sobre los que quiero hacer predicciones

conn =  cx_Oracle.connect(user="pc",password="p201404",dsn="siatchdesa")
query = "select * from L_DATOS_PREDICCION" 
df_pred = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM

# Para concatenar todos los chunks en un solo DataFrame (si tienes suficiente RAM)
df_pred_full = pd.concat(df_pred, ignore_index=True)
conn.close()

C:\Users\pcablinski.RWD\AppData\Local\Temp\ipykernel_187824\3968896330.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pred = pd.read_sql(query, conn, chunksize=1000000)  # Lee en chunks para no llenar la RAM


In [18]:
# Con el modelo entrenado, hacemos predicciones 
X_pred = df_pred_full[['PERIODO', 'CUSTOMER_ID', 'PRODUCT_ID']]
predictions = model.predict(X_pred)
# Agregar las predicciones al DataFrame original
df_pred_full['PREDICCIONES'] = predictions
# Imprimir las primeras filas del DataFrame con las predicciones
print(df_pred_full.head())
# Guardar el DataFrame con las predicciones en un archivo CSV
df_pred_full.to_csv('predicciones.csv', index=False)
# Imprimir el número de filas y columnas del DataFrame con las predicciones
print(f"Número de filas: {df_pred_full.shape[0]}, Número de columnas: {df_pred_full.shape[1]} con predicciones.")
# Guardar el modelo entrenado
model.save_model('modelo_regresion.txt')


   PERIODO  PRODUCT_ID  CUSTOMER_ID  TN_PREDICCION  PREDICCIONES
0   201912       20003        10001     132.089004    135.909964
1   201912       20027        10001      14.894868     16.940604
2   201912       20059        10001      10.696551     15.218868
3   201912       20063        10001      10.437913     12.136348
4   201912       20074        10001       9.246648     10.172317
Número de filas: 333840, Número de columnas: 5 con predicciones.


In [19]:
# Con el DataFrame de predicción, actualizamos la base de datos
# el criterio es actualizar la tabla L_DATOS_PREDICCION con las nuevas predicciones
# la columnna PREDICCIONES se debe actualizar con los nuevos valores
# la clave primaria es (PERIODO, CUSTOMER_ID, PRODUCT_ID)
# Hacer commit cada 10000 filas para evitar problemas de memoria

# Conectar a la base de datos para actualizar los datos de predicción
conn = cx_Oracle.connect(user="pc", password="p201404", dsn="siatchdesa")
# Crear un cursor para ejecutar las actualizaciones
cursor = conn.cursor()
# Iterar sobre las filas del DataFrame con las predicciones
for index, row in df_pred_full.iterrows():
    periodo = row['PERIODO']
    customer_id = row['CUSTOMER_ID']
    product_id = row['PRODUCT_ID']
    prediccion = row['PREDICCIONES']
    
    # Actualizar la tabla L_DATOS_PREDICCION con la nueva predicción
    update_query = """
        UPDATE L_DATOS_PREDICCION
        SET TN_PREDICCION = :prediccion
        WHERE PERIODO = :periodo AND CUSTOMER_ID = :customer_id AND PRODUCT_ID = :product_id
    """
    cursor.execute(update_query, {'prediccion': prediccion, 'periodo': periodo, 'customer_id': customer_id, 'product_id': product_id})  
    # Hacer commit cada 10000 filas para evitar problemas de memoria
    if index % 10000 == 0:
        conn.commit()
        print(f"Actualizadas {index} filas de L_DATOS_PREDICCION con las nuevas predicciones.")
# Confirmar los cambios en la base de datos
conn.commit()
# Cerrar el cursor y la conexión
cursor.close()
conn.close()
# Imprimir mensaje de finalización
print("Actualización de la tabla L_DATOS_PREDICCION completada con las nuevas predicciones.")


Actualizadas 0 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 10000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 20000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 30000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 40000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 50000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 60000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 70000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 80000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 90000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 100000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 110000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 120000 filas de L_DATOS_PREDICCION con las nuevas predicciones.
Actualizadas 